In [ ]:
import tensorflow as tf
#import tensorflow.contrib.eager as tfe
import numpy as np

print(tf.__version__)
print(tf.__git_version__)

2.2.0
v2.2.0-0-g2b96f3662b


In [ ]:
# set data dimensions
K = 3
V = 5
D = 100
N = 100

# set the seed
np.random.seed(2014)

# beta prior parameters
eta = np.ones(V) * 1e-1

# beta profiles
beta = np.random.dirichlet(alpha=eta, size=K)

# theta prior parameters
alpha = np.ones(K) * 1e-1
# alpha[0] = 10

# document's prior topic allocation
theta = np.random.dirichlet(alpha=alpha, size=D)

# word's topic membership
z = [np.random.choice(K, size=N, replace=True, p=theta[d, :]) for d in range(D)]
z = np.vstack(z)

# actual words and counts
w = [np.array([np.random.choice(V, size=1, p=beta[k,:])[0] for k in z[d, :]]  + list(range(V))) for d in range(D)]
nw = [np.unique(w[d], return_counts=True)[1] for d in range(D)]
nw = np.vstack(nw)
w = np.vstack(w)

nw = tf.convert_to_tensor(nw, dtype=tf.float32)
nw = tf.Variable(initial_value=tf.transpose(nw),
                 name="nw_vd")
# nw

In [ ]:
nw

<tf.Variable 'nw_vd:0' shape=(5, 100) dtype=float32, numpy=
array([[100.,  95.,  91.,  18.,  77.,  72.,  91.,  33.,  92.,  67.,   4.,
        101.,  93.,  94., 101.,  87.,   1.,  59., 101.,  98.,  98.,   2.,
         89.,  99.,  94.,  99.,  84., 100.,  97.,  39.,   1.,  99.,  99.,
         97.,  95.,  90.,  96.,  90., 101.,  61.,   6.,  99.,  92., 100.,
         99.,  98.,   1.,  97.,  80., 100.,  95.,  98.,  46.,   1.,  59.,
         96.,  97.,  74.,  64.,   6.,  97.,  26.,  87.,  92.,  98.,  98.,
         93.,  69.,  70.,  99.,  81.,  93.,  56.,  92.,  95., 101.,  96.,
         95.,  51., 101.,   6.,  62.,  92.,   1.,  97.,  12.,  91.,   1.,
         95.,  20., 100.,   2.,  92.,   4.,  91.,  78., 100.,  77.,  95.,
         93.],
       [  1.,   2.,   2.,   1.,   1.,   3.,   5.,   2.,   1.,   2.,   1.,
          1.,   3.,   3.,   1.,   2.,   1.,   1.,   1.,   1.,   1.,   1.,
          1.,   1.,   3.,   1.,   4.,   1.,   2.,   1.,   1.,   1.,   1.,
          3.,   2.,   3.,   2.,   1.,

In [ ]:
# initialize LDA parameters
def initialize_variables(K, V, D, alpha=1e-1, eta=1e-1, seed=2014):
    """
    Initialize parameters of LDA model returning adequate Tensors.

    args:

        K (int): number of LDA components
        V (int): vocabulary size
        D (int): number of documents
        alpha (float): hyperparameter for theta prior
        eta (float): hyperparameter for beta prior


    returns:

        eta: [V] tensor with prior parameters (alpha) for beta
        lambda: [K, V] tensor with posterior word distribution per class
        phi: [K, V, D] tensor with vocabulary membership per document
        gamma: [K, D] tensor

    """
    tf.random.set_seed(seed)
    eta = tf.Variable(initial_value=tf.ones(V) * eta,
                       name="eta_v")
    alpha = tf.Variable(initial_value=tf.ones(K) * alpha,
                         name="alpha_k")
    lam = tf.Variable(initial_value=tf.abs(tf.random.normal(shape=(K, V))),
                       name="lambda_kv")

    phi = tf.Variable(initial_value=tf.random.normal(shape=(K, V, D)),
                       name="phi_kvd")
    tf.compat.v1.assign(ref=phi, value=tf.nn.softmax(phi, axis=0))

    gamma = tf.Variable(initial_value=tf.abs(tf.random.normal(shape=(K, D))),
                        name="gamma_kd")

    e_log_beta = tf.Variable(initial_value=tf.abs(tf.random.normal(shape=(K, V, D))) * .0,
                        name="e_log_beta_kvd")

    e_log_theta = tf.Variable(initial_value=tf.abs(tf.random.normal(shape=(K, V, D))) * .0,
                        name="e_log_theta_kvd")

    return eta, alpha, lam, phi, gamma, e_log_beta, e_log_theta

# test
eta, alpha, lam, phi, gamma, e_log_beta, e_log_theta = initialize_variables(K, V, D)

In [ ]:
# lambda update
def update_lambda(lam, eta, phi, nw):

    K = lam.shape.as_list()[0]
    for k in range(K):
        tf.compat.v1.scatter_update(ref=lam,
                  indices=k,
                  updates=tf.reduce_sum(tf.multiply(phi[k], nw), axis=1) + eta)

    return lam

# test
# update_lambda(lam, eta, phi, nw)
# print(lam)

In [ ]:
# gamma update
def update_gamma(gamma, alpha, phi, nw):

    K = gamma.shape.as_list()[0]
    for k in range(K):
        tf.compat.v1.scatter_update(ref=gamma,
                  indices=k,
                  updates=tf.reduce_sum(tf.multiply(phi[k], nw), axis=0) + alpha[k])


    return gamma

tmp = gamma.value()
update_gamma(gamma, alpha, phi, nw)
print(gamma)

<tf.Variable 'gamma_kd:0' shape=(3, 100) dtype=float32, numpy=
array([[33.179443 , 28.432518 , 12.622646 , 17.165543 , 15.577526 ,
        10.490307 , 11.648556 , 32.405262 , 35.44801  , 46.716198 ,
         9.907303 , 71.49604  , 45.32922  , 27.23914  , 67.28874  ,
        49.506325 , 80.40026  , 33.695103 , 95.63079  , 70.015205 ,
        16.508184 , 20.57544  , 20.053913 , 26.896889 , 32.449123 ,
        16.200172 , 34.788353 , 68.103065 ,  9.126976 , 22.359228 ,
         4.256766 , 18.665571 , 22.737452 , 61.869286 , 42.570847 ,
         9.795437 , 14.991373 , 52.058765 , 35.303062 , 33.66349  ,
        35.514534 , 62.23464  , 13.184516 , 84.8964   , 87.078835 ,
        39.539482 ,  9.8654995, 59.58053  , 47.186913 ,  8.718346 ,
        62.436123 , 50.3837   , 40.483505 , 25.55224  , 34.0292   ,
        29.543427 ,  6.3052363, 44.445526 , 12.977353 , 18.668716 ,
        77.82936  , 61.49327  , 38.4598   , 68.482994 , 45.963047 ,
        95.63182  , 55.1969   , 37.85514  , 15.071856

In [ ]:
tmp

<tf.Tensor: shape=(3, 100), dtype=float32, numpy=
array([[0.856814  , 0.17276734, 2.0289466 , 0.36276183, 1.5049754 ,
        0.01352629, 0.8085556 , 0.05183626, 0.1407165 , 0.2526934 ,
        1.7273577 , 0.15571633, 0.38100538, 0.26664335, 0.9372891 ,
        0.5413353 , 1.3022376 , 0.1799982 , 1.3224765 , 0.13220988,
        0.46405008, 1.3386966 , 1.1340383 , 0.9544467 , 1.0398598 ,
        0.45572257, 0.41327694, 0.42305326, 1.7513535 , 1.2226332 ,
        1.3665848 , 0.69009304, 0.19026843, 0.9641658 , 1.1776    ,
        1.1071438 , 0.13112977, 0.8296427 , 0.79165363, 0.07314051,
        1.2809435 , 1.5483049 , 0.44105747, 0.32301107, 0.6834025 ,
        1.708542  , 0.08623107, 0.2677692 , 1.0648154 , 0.91856754,
        1.791696  , 0.58421534, 0.98128223, 0.99504936, 1.5844259 ,
        0.58608973, 0.20972344, 0.66854095, 0.9684806 , 0.9942798 ,
        0.34617952, 1.2616568 , 0.36821425, 0.5178254 , 0.6032014 ,
        0.25295106, 0.5769268 , 0.5076303 , 3.0258594 , 0.01671933

In [ ]:
def update_e_log_beta(e_log_beta, lam):

    K = lam.shape.as_list()[0]
    for k in range(K):
        tf.compat.v1.scatter_update(ref=e_log_beta,
                  indices=k,
                  updates=tf.tile(tf.expand_dims(tf.compat.v1.digamma(lam[k]) - tf.compat.v1.digamma(tf.reduce_sum(lam[k])), axis=1), multiples=[1, D]))

    return e_log_beta

print(e_log_beta)
update_e_log_beta(e_log_beta, lam);
print(e_log_beta)

<tf.Variable 'e_log_beta_kvd:0' shape=(3, 5, 100) dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)>
<tf.Variable 'e_log_beta_kvd:0' shape=(3, 5, 100) dtype=float32, numpy=
array([[[ -4.3613567,  -4.3613567,  -4.3613567, ...,  -4.3613567,
          -4.3613567,  -4.3613567],
        [ -1.004497 ,  -1.004497 ,  -1.004497 , ...,  -1.004497 ,
          -1.004497 ,  -1.004497 ],
        [ -3.9851837,  -3.9851837,  -3.985183

In [ ]:
def update_e_log_theta(e_log_theta, gamma):

    tf.compat.v1.assign(ref=e_log_theta,
              value=tf.tile(tf.expand_dims(tf.compat.v1.digamma(gamma) - tf.compat.v1.digamma(tf.reduce_sum(gamma, axis=0)), axis=1), multiples=[1, V, 1]))

    return e_log_theta

update_e_log_theta(e_log_theta, gamma)

<tf.Variable 'e_log_theta_kvd:0' shape=(3, 5, 100) dtype=float32, numpy=
array([[[-1.1652718 , -1.3222127 , -2.1566987 , ..., -0.71888995,
         -2.0216992 , -2.9808097 ],
        [-1.1652718 , -1.3222127 , -2.1566987 , ..., -0.71888995,
         -2.0216992 , -2.9808097 ],
        [-1.1652718 , -1.3222127 , -2.1566987 , ..., -0.71888995,
         -2.0216992 , -2.9808097 ],
        [-1.1652718 , -1.3222127 , -2.1566987 , ..., -0.71888995,
         -2.0216992 , -2.9808097 ],
        [-1.1652718 , -1.3222127 , -2.1566987 , ..., -0.71888995,
         -2.0216992 , -2.9808097 ]],

       [[-2.0262153 , -0.52194834, -0.5760493 , ..., -1.0551589 ,
         -0.36708975, -1.6438904 ],
        [-2.0262153 , -0.52194834, -0.5760493 , ..., -1.0551589 ,
         -0.36708975, -1.6438904 ],
        [-2.0262153 , -0.52194834, -0.5760493 , ..., -1.0551589 ,
         -0.36708975, -1.6438904 ],
        [-2.0262153 , -0.52194834, -0.5760493 , ..., -1.0551589 ,
         -0.36708975, -1.6438904 ],
       

In [ ]:
e_log_theta

<tf.Variable 'e_log_theta_kvd:0' shape=(3, 5, 100) dtype=float32, numpy=
array([[[-1.1652718 , -1.3222127 , -2.1566987 , ..., -0.71888995,
         -2.0216992 , -2.9808097 ],
        [-1.1652718 , -1.3222127 , -2.1566987 , ..., -0.71888995,
         -2.0216992 , -2.9808097 ],
        [-1.1652718 , -1.3222127 , -2.1566987 , ..., -0.71888995,
         -2.0216992 , -2.9808097 ],
        [-1.1652718 , -1.3222127 , -2.1566987 , ..., -0.71888995,
         -2.0216992 , -2.9808097 ],
        [-1.1652718 , -1.3222127 , -2.1566987 , ..., -0.71888995,
         -2.0216992 , -2.9808097 ]],

       [[-2.0262153 , -0.52194834, -0.5760493 , ..., -1.0551589 ,
         -0.36708975, -1.6438904 ],
        [-2.0262153 , -0.52194834, -0.5760493 , ..., -1.0551589 ,
         -0.36708975, -1.6438904 ],
        [-2.0262153 , -0.52194834, -0.5760493 , ..., -1.0551589 ,
         -0.36708975, -1.6438904 ],
        [-2.0262153 , -0.52194834, -0.5760493 , ..., -1.0551589 ,
         -0.36708975, -1.6438904 ],
       

In [ ]:
e_log_beta

<tf.Variable 'e_log_beta_kvd:0' shape=(3, 5, 100) dtype=float32, numpy=
array([[[ -4.3613567,  -4.3613567,  -4.3613567, ...,  -4.3613567,
          -4.3613567,  -4.3613567],
        [ -1.004497 ,  -1.004497 ,  -1.004497 , ...,  -1.004497 ,
          -1.004497 ,  -1.004497 ],
        [ -3.9851837,  -3.9851837,  -3.9851837, ...,  -3.9851837,
          -3.9851837,  -3.9851837],
        [ -3.3059347,  -3.3059347,  -3.3059347, ...,  -3.3059347,
          -3.3059347,  -3.3059347],
        [ -2.9363658,  -2.9363658,  -2.9363658, ...,  -2.9363658,
          -2.9363658,  -2.9363658]],

       [[ -2.9444265,  -2.9444265,  -2.9444265, ...,  -2.9444265,
          -2.9444265,  -2.9444265],
        [ -3.2798169,  -3.2798169,  -3.2798169, ...,  -3.2798169,
          -3.2798169,  -3.2798169],
        [ -1.8179834,  -1.8179834,  -1.8179834, ...,  -1.8179834,
          -1.8179834,  -1.8179834],
        [ -0.9550425,  -0.9550425,  -0.9550425, ...,  -0.9550425,
          -0.9550425,  -0.9550425],
        

In [ ]:
import time
start = time.time()

def update_phi(e_log_beta, e_log_theta):
    tf.compat.v1.assign(ref=phi,
              value=e_log_beta + e_log_theta)
    tf.compat.v1.assign(ref=phi, value=tf.nn.softmax(logits=phi, axis=0))
    return phi


update_phi(e_log_beta, e_log_theta)

end = time.time()
print(end - start)
print(phi)

NameError: ignored